<a href="https://colab.research.google.com/github/lujio20/EEG-RealTime-Signal-Analysis/blob/main/log%26_move_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

import mne
import pandas as pd
import numpy as np
from scipy.signal import welch
from google.colab import drive
import os

# 1. ربط الدرايف
drive.mount('/content/drive')

# 2. تحديد المسارات
raw_folder = "/content/drive/MyDrive/NeuroGift_Project/Raw_Data_EDF/"
save_folder = "/content/drive/MyDrive/NeuroGift_Project/Processed_Data/"

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"✅ تم إنشاء مجلد الحفظ: {save_folder}")
else:
    print(f"✅ مجلد الحفظ موجود: {save_folder}")

Mounted at /content/drive
✅ مجلد الحفظ موجود: /content/drive/MyDrive/NeuroGift_Project/Processed_Data/


In [ ]:
def process_smart_files(file_list, task_label):
    print(f"\n🚀 بدء معالجة {len(file_list)} ملفات لتصنيف: [{task_label}]...")

    all_rows = []

    # --- 🧠 إعدادات الذكاء حسب نوع المهمة ---
    if task_label == "Kinesthetic":
        target_channels = ['C3', 'C4', 'Cz']
        l_freq, h_freq = 8., 30.
        print("   ⚙️ وضع الذكاء الحركي: مفعل (Filters: 8-30Hz | Chans: C3,C4,Cz)")
    else:
        target_channels = ['Fp1', 'Fp2', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
        l_freq, h_freq = 1., 40.
        print("   ⚙️ وضع عام: مفعل (Filters: 1-40Hz | All Channels)")
    # ---------------------------------------

    for filename in file_list:
        file_path = raw_folder + filename
        print(f"   -> جاري معالجة الملف: {filename}...")

        try:
            raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)


            new_names = {ch: ch.split(' ')[-1].split('-')[0].strip('.') for ch in raw.ch_names}
            raw.rename_channels(new_names)

            # 3. اختيار القنوات المناسبة فقط
            available_chans = [ch for ch in target_channels if ch in raw.ch_names]

            if len(available_chans) == 0:
                print(f"      ⚠️ تخطي: لم يتم العثور على القنوات المطلوبة. القنوات المتاحة: {raw.ch_names[:5]}...")
                continue

            raw.pick_channels(available_chans)

            # 4. الفلتر
            raw.filter(l_freq, h_freq, fir_design='firwin', verbose=False)

            # 5. التقسيم (Epochs)
            epochs = mne.make_fixed_length_epochs(raw, duration=1.0, overlap=0.0, verbose=False)
            data = epochs.get_data(copy=True)

            # 6. استخراج السمات (PSD)
            fs = raw.info['sfreq']

            for i in range(len(data)):
                row = {'Label': task_label}
                for ch_idx, ch_name in enumerate(available_chans):
                    signal = data[i, ch_idx, :]
                    freqs, psd = welch(signal, fs=fs, nperseg=128)

                    # Alpha/Mu (8-13 Hz)
                    row[f"{ch_name}_Alpha"] = np.mean(psd[(freqs >= 8) & (freqs <= 13)])
                    # Beta (13-30 Hz)
                    row[f"{ch_name}_Beta"] = np.mean(psd[(freqs >= 13) & (freqs <= 30)])

                all_rows.append(row)

        except Exception as e:
            print(f"❌ خطأ في الملف {filename}: {e}")
            continue

    # الحفظ النهائي
    if len(all_rows) > 0:
        df = pd.DataFrame(all_rows).fillna(0)
        save_path = save_folder + f"{task_label}_Processed.csv"
        df.to_csv(save_path, index=False)
        print(f"✅ تم الحفظ بنجاح: {task_label}_Processed.csv (عدد العينات: {len(df)})")
    else:
        print(f"❌ لم يتم استخراج بيانات لـ {task_label}")

In [ ]:
# --- قوائم الملفات ---

logical_files = ["Subject00_2.edf", "Subject01_2.edf", "Subject02_2.edf"]
process_smart_files(logical_files, "Logical")

kinesthetic_files = ["S001R04.edf", "S002R04.edf", "S003R04.edf"]
process_smart_files(kinesthetic_files, "Kinesthetic")

print("-" * 30)
print("🎉 المهمة اكتملت يا بطلة! البيانات جاهزة للمرحلة القادمة.")


🚀 بدء معالجة 3 ملفات لتصنيف: [Logical]...
   ⚙️ وضع عام: مفعل (Filters: 1-40Hz | All Channels)
   -> جاري معالجة الملف: Subject00_2.edf...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 62 events and 500 original time points ...
0 bad epochs dropped
   -> جاري معالجة الملف: Subject01_2.edf...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 62 events and 500 original time points ...
0 bad epochs dropped
   -> جاري معالجة الملف: Subject02_2.edf...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Using data from preloaded Raw for 62 events and 500 original time points ...
0 bad epochs dropped
✅ تم الحفظ بنجاح: Logical_Processed.csv (عدد العينات: 186)

🚀 بدء معالجة 3 ملفات لتصنيف: [Kinesthetic]...
   ⚙️ وضع الذكاء الحركي: مفعل (Filters: 8-30Hz | Chans: C3,C4,Cz)
   -> جاري معالجة الملف: S001R04.edf...
NOTE: pick_channels() is a

In [ ]:
# كود التشخيص السريع
# نختار ملف واحد من ملفات الحركة
test_file = raw_folder + "S001R04.edf"

try:
    raw_test = mne.io.read_raw_edf(test_file, preload=False, verbose=False)
    print("📢 أسماء القنوات الحقيقية الموجودة في الملف هي:")
    print(raw_test.ch_names)
except Exception as e:
    print(f"❌ لم أستطع قراءة الملف: {e}")

📢 أسماء القنوات الحقيقية الموجودة في الملف هي:
['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
